# Creating a simple CNN architecture for adversarial training
Before we can perform an adversarial attack, we first need to implement our CNN architecture.

Once our architecture is implemented, we’ll train it on the MNIST dataset, evaluate it, generate a set of adversarial images using the FGSM, and re-evaluate it, thereby demonstrating the impact adversarial images have on accuracy.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MSE
import tensorflow as tf
import numpy as np

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

def generate_image_adversary(model, image, label, eps=2 / 255.0):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
        loss = MSE(label, pred)
        gradient = tape.gradient(loss, image)
        sign = tf.sign(gradient)
        adversary = (image + (sign * eps)).numpy()
        return adversary

# Train Target Model

This block need to run once before Applying FGSM Attack

In [42]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one-hot encode our labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# train the simple CNN on MNIST
print("[INFO] training network...")
print(trainX.shape)
model.fit(trainX, trainY,
          validation_data=(testX, testY),
          batch_size=64,
          epochs=10,
          verbose=1)

# make predictions on the testing set for the model trained on
# non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))
model.save("saved_model/TargetCNN")


[INFO] loading MNIST dataset...
[INFO] compiling model...
[INFO] training network...
(60000, 28, 28, 1)
Epoch 1/10


/opt/homebrew/Caskroom/miniforge/base/envs/cv/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


938/938 [==============================] - 6s 7ms/step - loss: 0.2043 - accuracy: 0.9391 - val_loss: 0.0911 - val_accuracy: 0.9714
Epoch 2/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0825 - accuracy: 0.9751 - val_loss: 0.0498 - val_accuracy: 0.9833
Epoch 3/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0637 - accuracy: 0.9802 - val_loss: 0.0489 - val_accuracy: 0.9856
Epoch 4/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0512 - accuracy: 0.9841 - val_loss: 0.0368 - val_accuracy: 0.9883
Epoch 5/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0430 - accuracy: 0.9864 - val_loss: 0.0346 - val_accuracy: 0.9887
Epoch 6/10
938/938 [==============================] - 6s 7ms/step - loss: 0.0345 - accuracy: 0.9886 - val_loss: 0.0331 - val_accuracy: 0.9893
Epoch 7/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0313 - accuracy: 0.9900 - val_loss: 0.0426 - val_accuracy: 0.9864
Epoch 8/10
938/93

In [5]:
def FGSM_train(model, ratio, eps):

    a_trainX = []
    a_trainY = []
    for i in np.random.choice(np.arange(0, len(trainX)), size=(int(len(trainX) * ratio),)):
        # grab the current image and label
        image = trainX[i]
        label = trainY[i]
        # generate an image adversary for the current image and make
        # a prediction on the adversary
        adversary = generate_image_adversary(model, image.reshape(1, 28, 28, 1), label, eps=eps)
        pred = model.predict(adversary)

        a_trainX.append(adversary)
        a_trainY.append(pred)


        # print(pred.argmax(), label.argmax())
        # scale both the original image and adversary to the range
        # [0, 255] and convert them to an unsigned 8-bit integers
        adversary = adversary.reshape((28, 28)) * 255
        adversary = np.clip(adversary, 0, 255).astype("uint8")
        image = image.reshape((28, 28)) * 255
        image = image.astype("uint8")

    a_trainX = np.array(a_trainX)
    a_trainY = np.array(a_trainY)

    a_trainX = a_trainX.squeeze(axis=1)
    a_trainY = a_trainY.squeeze(axis=1)

    model.fit(a_trainX, a_trainY,
              validation_data=(testX, testY),
              batch_size=64,
              epochs=10,
              verbose=1)
    (loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
    return loss, acc



In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np

print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one-hot encode our labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)


ratios = [0.2, 0.5, 0.7]
eps = [0.01, 0.001, 0.0001, 0.00001, 0.000001]

f = open('fgsm-res.csv', 'w')

for r in ratios:
    for e in eps:
        model = tf.keras.models.load_model('saved_model/TargetCNN')
        loss, acc = FGSM_train(model, r, e)
        f.write('%s, %s\n' % (loss, acc))

f.close()

[INFO] loading MNIST dataset...
Epoch 1/10
188/188 [==============================] - 2s 8ms/step - loss: 0.1777 - accuracy: 0.9587 - val_loss: 0.0420 - val_accuracy: 0.9872
Epoch 2/10
188/188 [==============================] - 1s 7ms/step - loss: 0.1622 - accuracy: 0.9649 - val_loss: 0.0430 - val_accuracy: 0.9863
Epoch 3/10
188/188 [==============================] - 1s 8ms/step - loss: 0.1528 - accuracy: 0.9680 - val_loss: 0.0340 - val_accuracy: 0.9886
Epoch 4/10
188/188 [==============================] - 1s 8ms/step - loss: 0.1465 - accuracy: 0.9689 - val_loss: 0.0423 - val_accuracy: 0.9859
Epoch 5/10
188/188 [==============================] - 1s 8ms/step - loss: 0.1358 - accuracy: 0.9732 - val_loss: 0.0399 - val_accuracy: 0.9872
Epoch 6/10
188/188 [==============================] - 1s 8ms/step - loss: 0.1326 - accuracy: 0.9761 - val_loss: 0.0389 - val_accuracy: 0.9874
Epoch 7/10
188/188 [==============================] - 2s 8ms/step - loss: 0.1309 - accuracy: 0.9754 - val_loss: 0.04